In [1]:
import numpy as np
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [33]:
# poems = data.poem_text.values

In [34]:
data = pd.read_csv('/content/drive/MyDrive/Barmagan/new_poetry_data.csv')

In [42]:
chosen_data = data.iloc[0:30000]
poems = list(chosen_data.refined_poem.values)

In [44]:
available_qafiyas = []
bad_poems=[]
for poem_id in range(0, len(poems)):

    try:
        idx = 1
        lines= poems[poem_id].split('\n')
        while(idx < len(lines)-2):
            qafiyas = []
            qafiya1 = lines[idx].split()[-1][-1]

            qafiyas.append(qafiya1)
            idx+=2
            qafiya2 = lines[idx].split()[-1][-1]
            qafiyas.append(qafiya2)
            new_poem = lines[idx-3:idx+1]
            qafiyas.append(new_poem)
            available_qafiyas.append(qafiyas)
            idx+=2
        
    except:
        bad_poems.append(poem_id)
        

In [46]:
ending11 = []
ending22 = []
new_poems2 = []

for poem_list in available_qafiyas:
    ending11.append(poem_list[0])
    ending22.append(poem_list[1])
    new_poems2.append(poem_list[2])

In [ ]:
chosen_data['ending11'] = ending11
chosen_data['ending22'] = ending22
chosen_data['new_poems2'] = new_poems2

In [50]:
chosen_data['new_poems2'] = chosen_data['new_poems2'].apply(reform_df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
chosen_data.sample()

,Unnamed: 0,ending1,ending2,poem,refined_poem
1470,1470,ِ,ِ,"['وَمَسكِنُ أَهلَها مِن بَطنِ جَزعٍ', 'تَبيضُ ...","['ومسكن أهلها من بطن جزع', 'تبيض به مصاييف الح..."


In [51]:
train_data =  chosen_data[(chosen_data['ending11'] == 'ا') |\
            (chosen_data['ending11'] == 'ب')|\
            (chosen_data['ending11'] == 'ر')|\
            (chosen_data['ending11'] == 'ه')|\
            (chosen_data['ending11'] == 'ل')|\
            (chosen_data['ending11'] == 'د')|\
            (chosen_data['ending11'] == 'م')|\
            (chosen_data['ending11'] == 'ي')|\
            (chosen_data['ending11'] == 'ء')|\
            (chosen_data['ending11'] == 'ن')|\
            (chosen_data['ending11'] == 'ت')|\
            (chosen_data['ending11'] == 'ح')|\
            (chosen_data['ending11'] == 'ق')]

In [52]:
train_data.sample()

,Unnamed: 0,ending1,ending2,poem,refined_poem,ending11,ending22,new_poems2
24387,24387,ِ,ِ,"['إِنَّ المَنِيَّةَ راوَحَت', 'ني يَومَ رُحتِ ...",إن المنية راوحت\nني يوم رحت مع الغياب\nإما ذهب...,ب,ب,إن المنية راوحت\nني يوم رحت مع الغياب\nإما ذهب...


In [54]:
from transformers import GPT2TokenizerFast, GPT2LMHeadModel

In [55]:
tokenizer = GPT2TokenizerFast.from_pretrained('aubmindlab/aragpt2-medium')

Downloading:   0%|          | 0.00/1.85M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.43M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/843 [00:00<?, ?B/s]

In [56]:
model = GPT2LMHeadModel.from_pretrained('aubmindlab/aragpt2-medium')

Downloading:   0%|          | 0.00/1.40G [00:00<?, ?B/s]

In [57]:
tokenizer.add_special_tokens({'sep_token': '[SEP]'})
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.add_special_tokens({'additional_special_tokens':['[NEWLINE]']})

1

In [58]:
model.resize_token_embeddings(len(tokenizer))

Embedding(64003, 1024)

In [14]:
chosen_data.sample()

,Unnamed: 0,Unnamed: 0.1,ending1,ending2,poem,refined_poem,ending11,ending22,new_poems2
9976,9976,9976,ُ,ُ,"['ينتحي من عِدائه في الثَّنِيَّا', 'تِ ولَحْبُ...",ينتحي من عدائه في الثنيا\nت ولحب الهدى له ملحو...,ب,ب,ينتحي من عدائه في الثنيا\nت ولحب الهدى له ملحو...


In [59]:
poems = list(chosen_data.new_poems2.values)
poems[0]

'خليلي عوجا فانظراني لعلني\nأسائل رسما قد عفا وتهدما\nبأوعس من ذات الحجى ما عرفته\nبعيد حصاة النفس إلا توهما\n'

In [60]:
ending11 = list(chosen_data.ending11.values)
ending22 = list(chosen_data.ending22.values)

In [61]:
def prepare_input(qafiya1, qafiya2, poem, tokenizer):
    input_sentences = []
    for q1, q2, p in zip(qafiya1, qafiya2, poem):
        # stacked_poem = ''
        # list_p = p.strip('][').split(', ')
        # for lineid in range(len(list_p)):
        #     stacked_poem += list_p[lineid].strip("'")+ '\n'
 

        line = tokenizer.bos_token + str(q1) + tokenizer.sep_token
        plines = p.split('\n')
        for linep in range(0,len(plines)):
          if(linep == 3):
            line += plines[linep] + tokenizer.eos_token
            break

          # if (linep >3):
          #   continue  
          else:
            line += plines[linep] + tokenizer.additional_special_tokens[0]
          
          
        input_sentences.append(line)

    return input_sentences

input_sentences = prepare_input(ending11, ending22, poems, tokenizer)

In [62]:
def get_max_length(input_sentences):
    max_len = 0
    for line in input_sentences:
        encoded_line = tokenizer.encode(line)
        encoded_len = len(encoded_line)
        max_len = max(max_len, encoded_len)
    
    return max_len
 
max_length = get_max_length(input_sentences)

In [ ]:
print(max_length)

53


In [63]:
import torch

In [64]:
def encode_input(input_sentences, tokenizer, max_len):
    masks = []
    ids = []
    for line in input_sentences:
        encoded_line = tokenizer.encode_plus(line, return_tensors = 'pt',\
                                       truncation = True,
                                       max_length = max_len,
                                       padding= 'max_length')
        ids.append(encoded_line['input_ids'])
        masks.append(encoded_line['attention_mask'])

    
    ids_tensor = torch.cat(ids, dim = 0)
    masks_tensor = torch.cat(masks, dim=0)
    
    return ids_tensor, masks_tensor

input_ids, attention_mask = encode_input(input_sentences, tokenizer, max_length)

In [ ]:
attention_mask.shape

torch.Size([10000, 50])

In [65]:
from torch.utils.data import DataLoader, TensorDataset, random_split
from transformers import AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm


In [66]:
device = torch.device('cuda')

In [ ]:
model.cuda()

In [68]:
dataset = TensorDataset(input_ids, attention_mask)


train_dataloader = DataLoader(dataset, shuffle = True, batch_size =8)



In [69]:
epochs = 5
total_steps = len(train_dataloader) * epochs
warmup = total_steps*0.2

optimizer = AdamW(model.parameters(), lr = 3e-5)
sched = get_linear_schedule_with_warmup(optimizer,
                              num_warmup_steps = warmup,
                              num_training_steps = total_steps)


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [70]:

idx = 0
for epoch in range(0, epochs):
  loop = tqdm(train_dataloader,total = len(train_dataloader))
  for batch in loop:
      optimizer.zero_grad()
      input_tensor = batch[0].to(device)
      attention_mask = batch[1].to(device)

      op = model(input_ids = input_tensor, attention_mask = attention_mask, labels = input_tensor)
      loss = op[0]
      loss.backward()
      optimizer.step()

      sched.step()
      loop.set_description(f'Epoch {epoch}')
      loop.set_postfix(loss= loss.item())

    


Epoch 4: 100%|██████████| 3750/3750 [15:31<00:00,  4.02it/s, loss=0.82]


In [71]:
model_name = 'GenerateByQafiya'
model.save_pretrained('/content/drive/MyDrive/Barmagan/' +model_name)

In [ ]:
test_batch = next(iter(test_dataloader))
test_input = test_batch[0]

In [74]:
prompt = '<|endoftext|> ا  <|endoftext|>'
encoded_prompt = tokenizer.encode(prompt, return_tensors = 'pt').to(device)
text = model.generate(encoded_prompt, max_length= 50, repetition_penalty = 4.0)
generated_text = tokenizer.decode(text[0], skip_special_tokens = False)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


In [75]:
print(generated_text)

<|endoftext|> ا  <|endoftext|>[PAD] الرزق المليكبه الملك المقتدىمن اتقى الله وليا[NEWLINE]والحمد في المكرمات من احرار ماكثنا به الهدباالمننشر الحقبلا رتقابا نبته فركب ربابه التربأ
